# Libraries

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Data 

In [2]:
phase_behaviour = pd.read_csv("Data/Unseen data predictions.csv")
phase_behaviour_unseen_PS = phase_behaviour[:814]
phase_behaviour_unseen_NPS = phase_behaviour[814:2035].reset_index()
phase_behaviour_unseen_PS_NPS =phase_behaviour[2035:].reset_index()

# Global helper values

In [4]:
# Number of data points per phase diagram
pb_points = 407 

# Investigated phase diagram area
area = np.array((0.4, 0.25, 0.4, 0.6, 0.375, 0, 0, 0.375, 0.6)).reshape(3,3)

# Colour map
color_map = {
    "Microemulsion":"green",
    "Nonmicroemulsion":"red",
    "True Positive":"green",
    "True Negative":"red",
    "False Positive":"blue",
    "False Negative":"blue"
}

# Plot (Unseen PS)

In [9]:
# Create figure with subplots
columns = 2
rows = 1

sub_specs = list()
for j in range(columns):
    sub_specs.append({'type':'scatterternary'})

specs = list()
for i in range(rows):
    specs.append(sub_specs)

fig = make_subplots(
    rows = rows, 
    cols = columns,
    specs = specs)

# Plot each phase diagram
for i in range(int(phase_behaviour_unseen_PS.shape[0]/pb_points)):

    # Helper values
    phase_diagram = i  # Phase diagram number
    j = phase_diagram*pb_points  # Starting point for each phase diagram dataset
    PS = phase_behaviour_unseen_PS['PS'][j] # Polar solvent (PS) name abbreviation
    NPS = phase_behaviour_unseen_PS['NPS'][j] # Non-polar solvent (NPS) name abbreviation

    row = int(np.floor(phase_diagram/columns)+1)
    column = np.remainder(phase_diagram,columns)+1

    # Show a sigle legend for related subplots
    legend = False
    if i == 0:
        legend = True
    
    for phase_behaviour_type, colour in color_map.items():
        # Get separate data series for microemulsions and non-micoremulsions and plot on ternary scatter plots
        if phase_behaviour_type == 'Microemulsion' or phase_behaviour_type == 'Nonmicroemulsion':
            SPC_conc = phase_behaviour_unseen_PS[j:j+pb_points].loc[phase_behaviour_unseen_PS['Experimental_phase_behaviour'] == phase_behaviour_type, 'SPC_concentration']
            NPS_conc = phase_behaviour_unseen_PS[j:j+pb_points].loc[phase_behaviour_unseen_PS['Experimental_phase_behaviour'] == phase_behaviour_type, 'NPS_concentration']
            PS_conc = phase_behaviour_unseen_PS[j:j+pb_points].loc[phase_behaviour_unseen_PS['Experimental_phase_behaviour'] == phase_behaviour_type, 'PS_concentration']
            
            fig.add_trace(go.Scatterternary({'mode':'markers', 'a':SPC_conc, 'b':NPS_conc, 'c':PS_conc,
                'marker_symbol':'square', 
                'marker_size':7.7,
                'marker_color':colour, 
                'marker_opacity':0.5,
                'legendgroup':phase_behaviour_type,  
                'showlegend':legend,
                'name':phase_behaviour_type,
                'hovertemplate':'PS: %{c}<br>SPC: %{a}<br> NPS: %{b}'}), 
                row = row, col = column)
        
        # Get separate data series for conformal predictions and plot on ternary scatter plots
        if phase_behaviour_type == 'True Positive' or phase_behaviour_type == 'True Negative' or phase_behaviour_type == 'False Negative' or phase_behaviour_type == 'False Positive' or phase_behaviour_type == 'Nonconforming':
            SPC_conc = phase_behaviour_unseen_PS[j:j+pb_points].loc[phase_behaviour_unseen_PS['Type_prediction'] == phase_behaviour_type, 'SPC_concentration']
            NPS_conc = phase_behaviour_unseen_PS[j:j+pb_points].loc[phase_behaviour_unseen_PS['Type_prediction'] == phase_behaviour_type, 'NPS_concentration']
            PS_conc = phase_behaviour_unseen_PS[j:j+pb_points].loc[phase_behaviour_unseen_PS['Type_prediction'] == phase_behaviour_type, 'PS_concentration']
            
            fig.add_trace(go.Scatterternary({'mode':'markers', 'a':SPC_conc, 'b':NPS_conc, 'c':PS_conc,
                'marker_symbol':'circle', 
                'marker_size': 5,
                'marker_color':colour, 
                'marker_opacity':1,
                'legendgroup':phase_behaviour_type,
                'showlegend':legend,  
                'name':phase_behaviour_type,
                'hovertemplate':'PS: %{c}<br>SPC: %{a}<br> NPS: %{b}'}), 
                row = row, col = column)
              
        # Plot investigated area boundary
        fig.add_trace(go.Scatterternary({'mode':'lines', 
        'a': area[0],
        'b': area[1],
        'c': area[2],
        'line_color':'#aaaaaa',
        'line_width':1, 
        'name': 'Boundary of investigated area',
        'showlegend':False,
        'hovertemplate':'PS: %{c}<br>SPC: %{a}<br> NPS: %{b}'}),
        row = row, col = column)

        # Update axis names
        fig.update_ternaries(aaxis_title = 'SPC (w/w)', 
                            baxis_title = str(NPS + ' (w/w)'), 
                            caxis_title = str(PS + ' (w/w)'), 
                            row = row, col = column)

    # Update axis names
    fig.update_ternaries(aaxis_title = 'SPC (w/w)', 
                        baxis_title = str(NPS + ' (w/w)'), 
                        caxis_title = str(PS + ' (w/w)'), 
                        row = row, col = column)
    
# Update all plots formating
fig.update_ternaries(
    aaxis_dtick = 0.1, baxis_dtick = 0.1, caxis_dtick = 0.1,
    aaxis_linecolor = '#aaaaaa', baxis_linecolor = '#aaaaaa', caxis_linecolor = '#aaaaaa',
    aaxis_gridcolor = '#cccccc', baxis_gridcolor = '#cccccc', caxis_gridcolor = '#cccccc',
    aaxis_tickfont_size = 6, baxis_tickfont_size = 6, caxis_tickfont_size = 6, 
    bgcolor = 'white')

fig.update_layout(
    autosize = False,
    width = 900,
    height = 400, 
    font_size = 10)

fig.write_html("Predictions (unseen PS).html")

# Plot (unseen NPS)

In [7]:
# Create figure with subplots
columns = 2
rows = 2

sub_specs = list()
for j in range(columns):
    sub_specs.append({'type':'scatterternary'})

specs = list()
for i in range(rows):
    specs.append(sub_specs)

fig = make_subplots(
    rows = rows, 
    cols = columns,
    specs = specs)

# Plot each phase diagram
for i in range(int(phase_behaviour_unseen_NPS.shape[0]/pb_points)):

    # Helper values
    phase_diagram = i  # Phase diagram number
    j = phase_diagram*pb_points  # Starting point for each phase diagram dataset
    PS = phase_behaviour_unseen_NPS['PS'][j] # Polar solvent (PS) name abbreviation
    NPS = phase_behaviour_unseen_NPS['NPS'][j] # Non-polar solvent (NPS) name abbreviation

    row = int(np.floor(phase_diagram/columns)+1)
    column = np.remainder(phase_diagram,columns)+1

    # Show a sigle legend for related subplots
    legend = False
    if i == 0:
        legend = True

    for phase_behaviour_type, colour in color_map.items():
        # Get separate data series for microemulsions and non-micoremulsions and plot on ternary scatter plots
        if phase_behaviour_type == 'Microemulsion' or phase_behaviour_type == 'Nonmicroemulsion':
            SPC_conc = phase_behaviour_unseen_NPS[j:j+pb_points].loc[phase_behaviour_unseen_NPS['Experimental_phase_behaviour'] == phase_behaviour_type, 'SPC_concentration']
            NPS_conc = phase_behaviour_unseen_NPS[j:j+pb_points].loc[phase_behaviour_unseen_NPS['Experimental_phase_behaviour'] == phase_behaviour_type, 'NPS_concentration']
            PS_conc = phase_behaviour_unseen_NPS[j:j+pb_points].loc[phase_behaviour_unseen_NPS['Experimental_phase_behaviour'] == phase_behaviour_type, 'PS_concentration']
            
            fig.add_trace(go.Scatterternary({'mode':'markers', 'a':SPC_conc, 'b':NPS_conc, 'c':PS_conc,
                'marker_symbol':'square', 
                'marker_size':7.7,
                'marker_color':colour, 
                'marker_opacity':0.5,
                'legendgroup':phase_behaviour_type,  
                'showlegend':legend,
                'name':phase_behaviour_type,
                'hovertemplate':'PS: %{c}<br>SPC: %{a}<br> NPS: %{b}'}), 
                row = row, col = column)
        
        # Get separate data series for conformal predictions and plot on ternary scatter plots
        if phase_behaviour_type == 'True Positive' or phase_behaviour_type == 'True Negative' or phase_behaviour_type == 'False Negative' or phase_behaviour_type == 'False Positive' or phase_behaviour_type == 'Nonconforming':
            SPC_conc = phase_behaviour_unseen_NPS[j:j+pb_points].loc[phase_behaviour_unseen_NPS['Type_prediction'] == phase_behaviour_type, 'SPC_concentration']
            NPS_conc = phase_behaviour_unseen_NPS[j:j+pb_points].loc[phase_behaviour_unseen_NPS['Type_prediction'] == phase_behaviour_type, 'NPS_concentration']
            PS_conc = phase_behaviour_unseen_NPS[j:j+pb_points].loc[phase_behaviour_unseen_NPS['Type_prediction'] == phase_behaviour_type, 'PS_concentration']
            
            fig.add_trace(go.Scatterternary({'mode':'markers', 'a':SPC_conc, 'b':NPS_conc, 'c':PS_conc,
                'marker_symbol':'circle', 
                'marker_size': 5,
                'marker_color':colour, 
                'marker_opacity':1,
                'legendgroup':phase_behaviour_type,
                'showlegend':legend,  
                'name':phase_behaviour_type,
                'hovertemplate':'PS: %{c}<br>SPC: %{a}<br> NPS: %{b}'}), 
                row = row, col = column)
              
        # Plot investigated area boundary
        fig.add_trace(go.Scatterternary({'mode':'lines', 
        'a': area[0],
        'b': area[1],
        'c': area[2],
        'line_color':'#aaaaaa',
        'line_width':1, 
        'name': 'Boundary of investigated area',
        'showlegend':False,
        'hovertemplate':'PS: %{c}<br>SPC: %{a}<br> NPS: %{b}'}),
        row = row, col = column)

        # Update axis names
        fig.update_ternaries(aaxis_title = 'SPC (w/w)', 
                            baxis_title = str(NPS + ' (w/w)'), 
                            caxis_title = str(PS + ' (w/w)'), 
                            row = row, col = column)
                            
    # Update axis names
    fig.update_ternaries(aaxis_title = 'SPC (w/w)', 
                        baxis_title = str(NPS + ' (w/w)'), 
                        caxis_title = str(PS + ' (w/w)'), 
                        row = row, col = column)
    
# Update all plots formating
fig.update_ternaries(
    aaxis_dtick = 0.1, baxis_dtick = 0.1, caxis_dtick = 0.1,
    aaxis_linecolor = '#aaaaaa', baxis_linecolor = '#aaaaaa', caxis_linecolor = '#aaaaaa',
    aaxis_gridcolor = '#cccccc', baxis_gridcolor = '#cccccc', caxis_gridcolor = '#cccccc',
    aaxis_tickfont_size = 6, baxis_tickfont_size = 6, caxis_tickfont_size = 6, 
    bgcolor = 'white')

fig.update_layout(
    autosize = False,
    width = 900,
    height = 800, 
    font_size = 10)

fig.write_html("Predictions (unseen NPS).html")

# Plot (unseen PS and NPS)

In [10]:
# Create figure with subplots
columns = 2
rows = 1

sub_specs = list()
for j in range(columns):
    sub_specs.append({'type':'scatterternary'})

specs = list()
for i in range(rows):
    specs.append(sub_specs)

fig = make_subplots(
    rows = rows, 
    cols = columns,
    specs = specs)

# Plot each phase diagram
for i in range(int(phase_behaviour_unseen_PS_NPS.shape[0]/pb_points)):

    # Helper values
    phase_diagram = i  # Phase diagram number
    j = phase_diagram*pb_points  # Starting point for each phase diagram dataset
    PS = phase_behaviour_unseen_PS_NPS['PS'][j] # Polar solvent (PS) name abbreviation
    NPS = phase_behaviour_unseen_PS_NPS['NPS'][j] # Non-polar solvent (NPS) name abbreviation

    row = int(np.floor(phase_diagram/columns)+1)
    column = np.remainder(phase_diagram,columns)+1

    # Show a sigle legend for related subplots
    legend = False
    if i == 0:
        legend = True

    for phase_behaviour_type, colour in color_map.items():
        # Get separate data series for microemulsions and non-micoremulsions and plot on ternary scatter plots
        if phase_behaviour_type == 'Microemulsion' or phase_behaviour_type == 'Nonmicroemulsion':
            SPC_conc = phase_behaviour_unseen_PS_NPS[j:j+pb_points].loc[phase_behaviour_unseen_PS_NPS['Experimental_phase_behaviour'] == phase_behaviour_type, 'SPC_concentration']
            NPS_conc = phase_behaviour_unseen_PS_NPS[j:j+pb_points].loc[phase_behaviour_unseen_PS_NPS['Experimental_phase_behaviour'] == phase_behaviour_type, 'NPS_concentration']
            PS_conc = phase_behaviour_unseen_PS_NPS[j:j+pb_points].loc[phase_behaviour_unseen_PS_NPS['Experimental_phase_behaviour'] == phase_behaviour_type, 'PS_concentration']
            
            fig.add_trace(go.Scatterternary({'mode':'markers', 'a':SPC_conc, 'b':NPS_conc, 'c':PS_conc,
                'marker_symbol':'square', 
                'marker_size':7.7,
                'marker_color':colour, 
                'marker_opacity':0.5,
                'legendgroup':phase_behaviour_type,  
                'showlegend':legend,
                'name':phase_behaviour_type,
                'hovertemplate':'PS: %{c}<br>SPC: %{a}<br> NPS: %{b}'}), 
                row = row, col = column)
        
        # Get separate data series for conformal predictions and plot on ternary scatter plots
        if phase_behaviour_type == 'True Positive' or phase_behaviour_type == 'True Negative' or phase_behaviour_type == 'False Negative' or phase_behaviour_type == 'False Positive' or phase_behaviour_type == 'Nonconforming':
            SPC_conc = phase_behaviour_unseen_PS_NPS[j:j+pb_points].loc[phase_behaviour_unseen_PS_NPS['Type_prediction'] == phase_behaviour_type, 'SPC_concentration']
            NPS_conc = phase_behaviour_unseen_PS_NPS[j:j+pb_points].loc[phase_behaviour_unseen_PS_NPS['Type_prediction'] == phase_behaviour_type, 'NPS_concentration']
            PS_conc = phase_behaviour_unseen_PS_NPS[j:j+pb_points].loc[phase_behaviour_unseen_PS_NPS['Type_prediction'] == phase_behaviour_type, 'PS_concentration']
            
            fig.add_trace(go.Scatterternary({'mode':'markers', 'a':SPC_conc, 'b':NPS_conc, 'c':PS_conc,
                'marker_symbol':'circle', 
                'marker_size': 5,
                'marker_color':colour, 
                'marker_opacity':1,
                'legendgroup':phase_behaviour_type,
                'showlegend':legend,  
                'name':phase_behaviour_type,
                'hovertemplate':'PS: %{c}<br>SPC: %{a}<br> NPS: %{b}'}), 
                row = row, col = column)
              
        # Plot investigated area boundary
        fig.add_trace(go.Scatterternary({'mode':'lines', 
        'a': area[0],
        'b': area[1],
        'c': area[2],
        'line_color':'#aaaaaa',
        'line_width':1, 
        'name': 'Boundary of investigated area',
        'showlegend':False,
        'hovertemplate':'PS: %{c}<br>SPC: %{a}<br> NPS: %{b}'}),
        row = row, col = column)

        # Update axis names
        fig.update_ternaries(aaxis_title = 'SPC (w/w)', 
                            baxis_title = str(NPS + ' (w/w)'), 
                            caxis_title = str(PS + ' (w/w)'), 
                            row = row, col = column)
                            
    # Update axis names
    fig.update_ternaries(aaxis_title = 'SPC (w/w)', 
                        baxis_title = str(NPS + ' (w/w)'), 
                        caxis_title = str(PS + ' (w/w)'), 
                        row = row, col = column)
    
# Update all plots formating
fig.update_ternaries(
    aaxis_dtick = 0.1, baxis_dtick = 0.1, caxis_dtick = 0.1,
    aaxis_linecolor = '#aaaaaa', baxis_linecolor = '#aaaaaa', caxis_linecolor = '#aaaaaa',
    aaxis_gridcolor = '#cccccc', baxis_gridcolor = '#cccccc', caxis_gridcolor = '#cccccc',
    aaxis_tickfont_size = 6, baxis_tickfont_size = 6, caxis_tickfont_size = 6, 
    bgcolor = 'white')

fig.update_layout(
    autosize = False,
    width = 900,
    height = 400, 
    font_size = 10)

fig.write_html("Predictions (unseen PS and NPS).html")